In [1]:
##Importamos las librerias correspondientes, antes se instalaron con pip install - pandas, numpy, matplot...etc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np  
import missingno as msno
import re
import ast
import json

sns.set_theme(color_codes= True)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option("max_colwidth", 100)

In [2]:
##Leemos los archivos csv con pandas
df_Movies = pd.read_csv('DATA/movies_dataset.csv')
df_credits = pd.read_csv('DATA/credits.csv')
df_Movies_credits = pd.read_csv('DATA/df_etl.csv')

C:\Users\Raksasa\AppData\Local\Temp\ipykernel_11976\2018209022.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_Movies = pd.read_csv('DATA/movies_dataset.csv')


FileNotFoundError: [Errno 2] No such file or directory: 'DATA/df_etl.csv'

In [ ]:
#Creamos las funciones necesarias para desanidar los datos anidados

def desanidar_columnas(dataframe, columnas):
    desanidados = {}
    for columna in columnas:
        nueva_columna = columna + "_desanidada"
        desanidados[nueva_columna] = []
        for valor in dataframe[columna]:
            desanidados[nueva_columna].append(desanidar_valor(valor))
    return desanidados


def desanidar_valor(valor):
    if isinstance(valor, dict):
        return desanidar_diccionario(valor)
    elif isinstance(valor, list):
        return desanidar_lista(valor)
    else:
        return valor

def desanidar_diccionario(diccionario):
    desanidado = {}
    for clave, valor in diccionario.items():
        if isinstance(valor, dict):
            desanidado[clave] = desanidar_diccionario(valor)
        elif isinstance(valor, list):
            desanidado[clave] = desanidar_lista(valor)
        else:
            desanidado[clave] = valor
    return desanidado

def desanidar_lista(lista):
    desanidado = []
    for elemento in lista:
        if isinstance(elemento, dict):
            desanidado.append(desanidar_diccionario(elemento))
        elif isinstance(elemento, list):
            desanidado.append(desanidar_lista(elemento))
        else:
            desanidado.append(elemento)
    return desanidado

#Creamos una función para agregar las columnas desanidadas

def agregar_columnas_desanidadas(df, columnas_anidadas):
    # Creamos una copia del DataFrame original
    df_modificado = df.copy()

    # Recorremos las columnas desanidadas y agregamos una a la vez al DataFrame copiado 'df_modificado'
    for columna in columnas_anidadas:
        valores = df_modificado[columna]
        df_modificado[columna + "_desanidada"] = pd.Series(valores)

    # Devolver el DataFrame modificado
    return df_modificado

def eliminar_columnas(dataframe, columnas):
    dataframe.drop(columns=columnas, inplace=True)
    return dataframe


#Creamos una función para transformar los datos y quitar el .0 de los valores de las columnas
def formatear_columnas(dataframe, *columnas):
    for columna in columnas:
        dataframe[columna] = dataframe[columna].fillna(0).astype(float).astype(int).astype(str)
        dataframe[columna] = dataframe[columna].str.replace('.0', '')
    return dataframe

In [ ]:
import ast

def desanidar_crew(crews):
    crews_id = []
    crews_department = []
    crews_gender = []
    crews_id_M = []
    crews_job = []
    crews_name = []
    crews_profile_path = []

    for crew in crews:
        crews_id.append(crew.get('credit_id'))
        crews_department.append(crew.get('department'))
        crews_gender.append(crew.get('gender'))
        crews_id_M.append(crew.get('id'))
        crews_job.append(crew.get('job'))
        crews_name.append(crew.get('name'))
        crews_profile_path.append(crew.get('profile_path'))

    return crews_id, crews_department, crews_gender, crews_id_M, crews_job, crews_name, crews_profile_path

def parse_crew_data(x):
    try:
        crews = ast.literal_eval(x)
        return desanidar_crew(crews)
    except (SyntaxError, ValueError):
        return ([], [], [], [], [], [], [])

generos_desanidados = df_Movies_credits['crew'].apply(parse_crew_data)

nuevo_df_movies = pd.DataFrame(generos_desanidados.tolist(), columns=['crews_id','crews_department','crews_gender','crews_id_M','crews_job','crews_name','crews_profile_path'])

nuevo_df_movies = nuevo_df_movies.apply(pd.Series.explode)
nuevo_df_movies = nuevo_df_movies.reset_index(drop=True)

nuevo_df_movies.head()


,crews_id,crews_department,crews_gender,crews_id_M,crews_job,crews_name,crews_profile_path
0,52fe4284c3a36847f8024f49,Directing,2,7879,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg
1,52fe4284c3a36847f8024f4f,Writing,2,12891,Screenplay,Joss Whedon,/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg
2,52fe4284c3a36847f8024f55,Writing,2,7,Screenplay,Andrew Stanton,/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg
3,52fe4284c3a36847f8024f5b,Writing,2,12892,Screenplay,Joel Cohen,/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg
4,52fe4284c3a36847f8024f61,Writing,0,12893,Screenplay,Alec Sokolow,/v79vlRYi94BZUQnkkyznbGUZLjT.jpg


In [ ]:
# Suponiendo que 'columnas_a_eliminar' es una lista con los nombres de las columnas a eliminar, para poder tener mas espacio
columnas_a_eliminar = ['crews_id','crews_department','crews_gender','crews_id_M','crews_profile_path']
nuevo_df_movies = nuevo_df_movies.drop(columnas_a_eliminar, axis=1)

In [ ]:
nuevo_df_movies.to_csv('DATA/df_crewfinal.csv', index=False)

## Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage.



In [ ]:
columnas_anidadas = ['genres', 'belongs_to_collection','production_companies','production_countries', 'spoken_languages']
desanidos = desanidar_columnas(df_Movies_credits, columnas_anidadas)

In [ ]:
df_movies1 = agregar_columnas_desanidadas(df_Movies_credits, columnas_anidadas)

In [ ]:
df_movies1 = eliminar_columnas(df_movies1, [ 'genres', 'belongs_to_collection', 'production_companies', 'production_countries', 'spoken_languages'])


In [ ]:
def formatear_columnas(dataframe, *columnas):
    for columna in columnas:
        # Replace NaN and inf with 0
        dataframe[columna] = dataframe[columna].fillna(0).replace([np.inf, -np.inf], 0)
        # Convert the column to float and then to integer
        dataframe[columna] = dataframe[columna].astype(float).astype(int).astype(str)
        dataframe[columna] = dataframe[columna].str.replace('.0', '')
    return dataframe

In [ ]:
df_movies1.head(1)

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,cast,crew,release_year,return,genres_desanidada,belongs_to_collection_desanidada,production_companies_desanidada,production_countries_desanidada,spoken_languages_desanidada
0,30000000,862.0,en,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear o...",21.95,1995-10-30,373554033,81,Released,[{'Sin Datos'}],Toy Story,7.7,5415,"[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender'...","[{'credit_id': '52fe4284c3a36847f8024f49', 'department': 'Directing', 'gender': 2, 'id': 7879, '...",1995,12.45,"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]","{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg',...","[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]","[{'iso_639_1': 'en', 'name': 'English'}]"


In [ ]:
df_movies1['crew'] = df_movies1['crew'].str.replace(r'\b\w{24}\b', '', regex=True)
df_movies1['release_date'] = pd.to_datetime(df_movies1['release_date'])
df_movies1.head(1)

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,cast,crew,release_year,return,genres_desanidada,belongs_to_collection_desanidada,production_companies_desanidada,production_countries_desanidada,spoken_languages_desanidada
0,30000000,862.0,en,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear o...",21.95,1995-10-30,373554033,81,Released,[{'Sin Datos'}],Toy Story,7.7,5415,"[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender'...","[{'credit_id': '', 'department': 'Directing', 'gender': 2, 'id': 7879, 'job': 'Director', 'name'...",1995,12.45,"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]","{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg',...","[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]","[{'iso_639_1': 'en', 'name': 'English'}]"


In [ ]:
# Expandir la columna "genres" en filas
genres_df = df_movies1.explode("genres_desanidada")

# Crear un nuevo DataFrame con las columnas deseadas
movie_genres_df = genres_df[["id", "genres_desanidada"]]

# Guardar el DataFrame en un archivo CSV
movie_genres_df.to_csv("DATA/movie_genres.csv", index=False)

In [ ]:
# Assuming df_genre is the DataFrame loaded from "DATA/movie_genres.csv"
df_genre = pd.read_csv("DATA/movie_genres.csv")

# Remove leading zeros and convert 'id' column to numeric, handling non-numeric values
df_genre['id'] = pd.to_numeric(df_genre['id'], errors='coerce').astype('Int64')

df_genre.head()

,id,genres_desanidada
0,862,"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"
1,8844,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]"
2,15602,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}]"
3,31357,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]"
4,11862,"[{'id': 35, 'name': 'Comedy'}]"


In [ ]:
import ast

def desanidar_generos(genres):
    genero_id = [genero['id'] for genero in genres]
    genero_nombre = [genero['name'] for genero in genres]
    return genero_id, genero_nombre

# Filtrar los valores no nulos en la columna 'genres_desanidada'
generos_desanidados = df_genre['genres_desanidada'].apply(lambda x: desanidar_generos(ast.literal_eval(x)) if pd.notna(x) else ([], []))

# Crear un nuevo DataFrame a partir de los géneros desanidados
nuevo_df = pd.DataFrame(generos_desanidados.tolist(), columns=['genero_id', 'genero_nombre'])

# Explotar las columnas para que cada fila tenga un solo género
nuevo_df = nuevo_df.explode('genero_id').reset_index(drop=True)

# Mostrar las primeras filas del nuevo DataFrame
nuevo_df.head(5)


,genero_id,genero_nombre
0,16,"[Animation, Comedy, Family]"
1,35,"[Animation, Comedy, Family]"
2,10751,"[Animation, Comedy, Family]"
3,12,"[Adventure, Fantasy, Family]"
4,14,"[Adventure, Fantasy, Family]"


In [ ]:
del df_genre['genres_desanidada']

In [ ]:
df_genre.head()

,id
0,862
1,8844
2,15602
3,31357
4,11862


In [ ]:
# Unir el DataFrame original con las columnas 'genero_id' y 'genero_nombre' del nuevo DataFrame
df = df_genre.join(nuevo_df[['genero_id', 'genero_nombre']])

In [ ]:
df.head()

,id,genero_id,genero_nombre
0,862,16,"[Animation, Comedy, Family]"
1,8844,35,"[Animation, Comedy, Family]"
2,15602,10751,"[Animation, Comedy, Family]"
3,31357,12,"[Adventure, Fantasy, Family]"
4,11862,14,"[Adventure, Fantasy, Family]"


In [ ]:
df.to_csv('DATA/movie_genres.csv', index=False)

In [ ]:
movie_colection = df_movies1[['id', 'belongs_to_collection_desanidada']].copy()

In [ ]:
# Remove leading zeros and convert 'id' column to numeric, handling non-numeric values
movie_colection['id'] = pd.to_numeric(movie_colection['id'], errors='coerce').astype('Int64')

In [ ]:
def obtener_valor(diccionario, clave):
    if pd.isnull(diccionario):
        return None
    if not isinstance(diccionario, dict):
        try:
            diccionario = json.loads(diccionario.replace("'", "\""))
        except (json.JSONDecodeError, AttributeError):
            return None
    if isinstance(diccionario, dict):
        return diccionario.get(clave)
    else:
        return None

# Aplicar la función a la columna "belongs_to_collection"
movie_colection['id_collect'] = df_movies1['belongs_to_collection_desanidada'].apply(lambda x: obtener_valor(x, 'id'))
movie_colection['name_collect'] = df_movies1['belongs_to_collection_desanidada'].apply(lambda x: obtener_valor(x, 'name'))
movie_colection['poster_collect'] = df_movies1['belongs_to_collection_desanidada'].apply(lambda x: obtener_valor(x, 'poster_path'))
movie_colection['backdrop_collect'] = df_movies1['belongs_to_collection_desanidada'].apply(lambda x: obtener_valor(x, 'backdrop_path'))

# Convertir la columna 'id_collect' a tipo entero con soporte para valores NaN
movie_colection['id_collect'] = movie_colection['id_collect'].astype(pd.Int64Dtype())

In [ ]:
movie_colection.head()

,id,belongs_to_collection_desanidada,id_collect,name_collect,poster_collect,backdrop_collect
0,862,"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg',...",10194,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg
1,8844,[{'Sin Datos'}],<NA>,None,None,None
2,15602,"{'id': 119050, 'name': 'Grumpy Old Men Collection', 'poster_path': '/nLvUdqgPgm3F85NMCii9gVFUcet...",119050,Grumpy Old Men Collection,/nLvUdqgPgm3F85NMCii9gVFUcet.jpg,/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg
3,31357,[{'Sin Datos'}],<NA>,None,None,None
4,11862,"{'id': 96871, 'name': 'Father of the Bride Collection', 'poster_path': '/nts4iOmNnq7GNicycMJ9pSA...",96871,Father of the Bride Collection,/nts4iOmNnq7GNicycMJ9pSAn204.jpg,/7qwE57OVZmMJChBpLEbJEmzUydk.jpg


In [ ]:
# Concatenar las columnas 'id_collect' y 'name_collect' de movie_colection a df_movies1
df_movies1 = pd.concat([df_movies1, movie_colection[['name_collect']]], axis=1)

In [ ]:
movies_country = df_movies1[['id', 'production_countries_desanidada']].copy()

In [ ]:
movies_country.head()

,id,production_countries_desanidada
0,862.0,"[{'iso_3166_1': 'US', 'name': 'United States of America'}]"
1,8844.0,"[{'iso_3166_1': 'US', 'name': 'United States of America'}]"
2,15602.0,"[{'iso_3166_1': 'US', 'name': 'United States of America'}]"
3,31357.0,"[{'iso_3166_1': 'US', 'name': 'United States of America'}]"
4,11862.0,"[{'iso_3166_1': 'US', 'name': 'United States of America'}]"


In [ ]:
tipos_de_datos = movies_country.dtypes
print(tipos_de_datos)


id                                 float64
production_countries_desanidada     object
dtype: object


In [ ]:
# Verificar el tipo de datos de la columna "id"
is_int64 = movies_country['id'].dtype == np.int64

if is_int64:
    print("Todos los valores de la columna 'id' son int64")
else:
    print("Algunos valores de la columna 'id' no son int64")

Algunos valores de la columna 'id' no son int64


In [ ]:
# Convertir la columna 'id' a tipo numérico int64 y eliminar filas no convertibles
movies_country['id'] = pd.to_numeric(movies_country['id'], errors='coerce', downcast='integer')
movies_country = movies_country.dropna(subset=['id'])

# Verificar si todas las filas de la columna 'id' son de tipo int64
if movies_country['id'].dtype == np.int64:
    print("Todos los valores de la columna 'id' son int64")
else:
    print("Algunas filas fueron eliminadas y los valores restantes en la columna 'id' son int64")
    
movies_country.head()

Algunas filas fueron eliminadas y los valores restantes en la columna 'id' son int64


,id,production_countries_desanidada
0,862.0,"[{'iso_3166_1': 'US', 'name': 'United States of America'}]"
1,8844.0,"[{'iso_3166_1': 'US', 'name': 'United States of America'}]"
2,15602.0,"[{'iso_3166_1': 'US', 'name': 'United States of America'}]"
3,31357.0,"[{'iso_3166_1': 'US', 'name': 'United States of America'}]"
4,11862.0,"[{'iso_3166_1': 'US', 'name': 'United States of America'}]"


In [ ]:
import ast

def parse_countries(x):
    try:
        return ast.literal_eval(x)
    except (SyntaxError, ValueError):
        return []

# Convertir las cadenas de texto en la columna "production_countries_desanidada" en listas de diccionarios
df_movies1['production_countries_desanidada'] = df_movies1['production_countries_desanidada'].apply(parse_countries)

# Crear columnas 'pc_iso', 'pc_name' en new_df
movies_country['id'] = df_movies1['id']
movies_country['country_iso'] = df_movies1['production_countries_desanidada'].apply(lambda x: ', '.join(pc['iso_3166_1'] for pc in x))
movies_country['name_country'] = df_movies1['production_countries_desanidada'].apply(lambda x: ', '.join(pc['name'] for pc in x))
# Imprimir los valores obtenidos
movies_country.head()

,id,production_countries_desanidada,country_iso,name_country
0,862.0,"[{'iso_3166_1': 'US', 'name': 'United States of America'}]",US,United States of America
1,8844.0,"[{'iso_3166_1': 'US', 'name': 'United States of America'}]",US,United States of America
2,15602.0,"[{'iso_3166_1': 'US', 'name': 'United States of America'}]",US,United States of America
3,31357.0,"[{'iso_3166_1': 'US', 'name': 'United States of America'}]",US,United States of America
4,11862.0,"[{'iso_3166_1': 'US', 'name': 'United States of America'}]",US,United States of America


In [ ]:
# Eliminar columna "production_countries_desanidada"
movies_country = movies_country.drop('production_countries_desanidada', axis=1)
# Imprimir los valores obtenidos
movies_country.head()

,id,country_iso,name_country
0,862.0,US,United States of America
1,8844.0,US,United States of America
2,15602.0,US,United States of America
3,31357.0,US,United States of America
4,11862.0,US,United States of America


In [ ]:
# Concatenar las columnas 'id_collect' y 'name_collect' de movie_colection a df_movies1
df_movies1 = pd.concat([df_movies1, movies_country[['name_country']]], axis=1)

In [ ]:
movies_companies = df_movies1[['id', 'production_companies_desanidada']].copy()
movies_companies.head()

,id,production_companies_desanidada
0,862.0,"[{'name': 'Pixar Animation Studios', 'id': 3}]"
1,8844.0,"[{'name': 'TriStar Pictures', 'id': 559}, {'name': 'Teitler Film', 'id': 2550}, {'name': 'Inters..."
2,15602.0,"[{'name': 'Warner Bros.', 'id': 6194}, {'name': 'Lancaster Gate', 'id': 19464}]"
3,31357.0,"[{'name': 'Twentieth Century Fox Film Corporation', 'id': 306}]"
4,11862.0,"[{'name': 'Sandollar Productions', 'id': 5842}, {'name': 'Touchstone Pictures', 'id': 9195}]"


In [ ]:
tipos_de_datos = movies_companies.dtypes
print(tipos_de_datos)

id                                 float64
production_companies_desanidada     object
dtype: object


In [ ]:
# Verificar el tipo de datos de la columna "id"
is_int64 = movies_companies['id'].dtype == np.int64

if is_int64:
    print("Todos los valores de la columna 'id' son int64")
else:
    print("Algunos valores de la columna 'id' no son int64")

Algunos valores de la columna 'id' no son int64


In [ ]:
# Convertir la columna 'id' a tipo numérico int64 y eliminar filas no convertibles
movies_companies['id'] = pd.to_numeric(movies_companies['id'], errors='coerce', downcast='integer')
movies_companies = movies_companies.dropna(subset=['id'])

# Verificar si todas las filas de la columna 'id' son de tipo int64
if movies_companies['id'].dtype == np.int64:
    print("Todos los valores de la columna 'id' son int64")
else:
    print("Algunas filas fueron eliminadas y los valores restantes en la columna 'id' son int64")

movies_companies.head()

Algunas filas fueron eliminadas y los valores restantes en la columna 'id' son int64


,id,production_companies_desanidada
0,862.0,"[{'name': 'Pixar Animation Studios', 'id': 3}]"
1,8844.0,"[{'name': 'TriStar Pictures', 'id': 559}, {'name': 'Teitler Film', 'id': 2550}, {'name': 'Inters..."
2,15602.0,"[{'name': 'Warner Bros.', 'id': 6194}, {'name': 'Lancaster Gate', 'id': 19464}]"
3,31357.0,"[{'name': 'Twentieth Century Fox Film Corporation', 'id': 306}]"
4,11862.0,"[{'name': 'Sandollar Productions', 'id': 5842}, {'name': 'Touchstone Pictures', 'id': 9195}]"


In [ ]:
# Convertir el valor del id a tipo numérico (flotante) utilizando pd.to_numeric
movies_companies['id'] = pd.to_numeric(movies_companies['id'], errors='coerce')

# Convertir los valores numéricos en la columna 'id' a enteros utilizando astype(int)
movies_companies['id'] = movies_companies['id'].astype(int)

In [ ]:
# Función para convertir cadenas en listas de diccionarios
def parse_companies(x):

    try:
        return ast.literal_eval(x)
    except (SyntaxError, ValueError):
        return []

# Convertir las cadenas de texto en la columna "production_companies" en listas de diccionarios
df_movies1['production_companies_desanidada'] = df_movies1['production_companies_desanidada'].apply(parse_companies)

# Crear columnas 'id', 'pc_id' y 'pc_name' en new_df
movies_companies['id'] = df_movies1['id']
movies_companies['company_id'] = df_movies1['production_companies_desanidada'].apply(lambda x: ', '.join(str(pc['id']) for pc in x))
movies_companies['name_company'] = df_movies1['production_companies_desanidada'].apply(lambda x: ', '.join(pc['name'] for pc in x))

movies_companies.head()

,id,production_companies_desanidada,company_id,name_company
0,862.0,"[{'name': 'Pixar Animation Studios', 'id': 3}]",3,Pixar Animation Studios
1,8844.0,"[{'name': 'TriStar Pictures', 'id': 559}, {'name': 'Teitler Film', 'id': 2550}, {'name': 'Inters...","559, 2550, 10201","TriStar Pictures, Teitler Film, Interscope Communications"
2,15602.0,"[{'name': 'Warner Bros.', 'id': 6194}, {'name': 'Lancaster Gate', 'id': 19464}]","6194, 19464","Warner Bros., Lancaster Gate"
3,31357.0,"[{'name': 'Twentieth Century Fox Film Corporation', 'id': 306}]",306,Twentieth Century Fox Film Corporation
4,11862.0,"[{'name': 'Sandollar Productions', 'id': 5842}, {'name': 'Touchstone Pictures', 'id': 9195}]","5842, 9195","Sandollar Productions, Touchstone Pictures"


In [ ]:
# Eliminar columna "production_countries_desanidada"
movies_companies = movies_companies.drop('production_companies_desanidada', axis=1)
# Imprimir los valores obtenidos
movies_companies.head()

,id,company_id,name_company
0,862.0,3,Pixar Animation Studios
1,8844.0,"559, 2550, 10201","TriStar Pictures, Teitler Film, Interscope Communications"
2,15602.0,"6194, 19464","Warner Bros., Lancaster Gate"
3,31357.0,306,Twentieth Century Fox Film Corporation
4,11862.0,"5842, 9195","Sandollar Productions, Touchstone Pictures"


In [ ]:
# Concatenar las columnas 'id_collect' y 'name_collect' de movie_colection a df_movies1
df_movies1 = pd.concat([df_movies1, movies_companies[['name_company']]], axis=1)

In [ ]:
df_movies1.to_csv('DATA/df_movies1.csv', index=False)